In [1]:
import pandas as pd 
import openai
import json

openai.organization = 'YOUR ORGANIZATION ID'
openai.api_key = 'YOUR API KEY'

Read and format data

In [2]:
df = pd.read_csv('data/fine_tune_hemolysis.csv')

# Training set
df_train = df[df['Set'] == 'training']
df_train = df_train[['Sequence', 'isNotHemolytic']]
df_train.columns = ['prompt', 'completion']
df_train = df_train[(df_train['completion'] == 0) | (df_train['completion'] == 1)].copy()
df_train.to_json('data/hemolysis_seq_train.jsonl', orient='records', lines=True)

# Test set
df_test = df[df['Set'] == 'test']
df_test = df_test[['Sequence', 'isNotHemolytic']]
df_test.columns = ['prompt', 'completion']
df_test = df_test[(df_test['completion'] == 0) | (df_test['completion'] == 1)].copy()
df_test.to_json('data/hemolysis_seq_test.jsonl', orient='records', lines=True)

Prepare data for submission with OpenAI API

In [ ]:
# openai tools fine_tunes.prepare_data -f hemolysis_seq_train.jsonl    
# openai tools fine_tunes.prepare_data -f hemolysis_seq_test.jsonl

Train the model

In [ ]:
# openai --api-key 'YOUR API KEY' api fine_tunes.create -t "hemolysis_seq_train_prepared.jsonl" -v "hemolysis_seq_test_prepared.jsonl" --compute_classification_metrics --classification_positive_class " 1" --model curie --suffix 'hemolysis' --n_epochs 10

In [ ]:
# openai --api-key 'YOUR API KEY' api fine_tunes.follow -i ft-sr7CTt4Id1zITladXjDjiMqm

Check results of classification

In [ ]:
# openai --api-key 'YOUR API KEY' api fine_tunes.results -i ft-sr7CTt4Id1zITladXjDjiMqm > curie_result_hemolysis.csv

In [2]:
results = pd.read_csv('data/curie_result_hemolysis.csv')
results.tail(1)

,step,elapsed_tokens,elapsed_examples,training_loss,training_sequence_accuracy,training_token_accuracy,validation_loss,validation_sequence_accuracy,validation_token_accuracy,classification/accuracy,classification/precision,classification/recall,classification/auroc,classification/auprc,classification/f1.0
8615,8616,329216,17232,0.003034,1.0,1.0,NaN,NaN,NaN,0.840445,0.821101,0.792035,0.891374,0.869639,0.806306
